In [ ]:
import quest
import holoviews as hv
import hvplot.pandas
import panel as pn

pn.extension()

# Query USGS Data Service for Data

In [ ]:
service_selector = quest.util.ServiceSelector(default='svc://usgs-nwis:iv')
pn.panel(service_selector)

In [ ]:
download_options = quest.api.get_download_options(service_selector.value, fmt='param')[service_selector.value]
download_options.parameter = download_options.param['parameter'].objects[0]
pn.panel(download_options)

In [ ]:
data = quest.api.get_data(
    service_uri=service_selector.value,
    search_filters={'bbox': [-91, 32.3, -90.8, 32.34]},
    download_options=download_options,
    collection_name='examples',
    use_cache=True,
    as_open_datasets=True,
    raise_on_error=True,
)[0]
data.head()

In [ ]:
data[download_options.parameter].hvplot()

# Run a Tool to Transform Downloaded Data

In [ ]:
tool = quest.tools.ts_resample
tool.dataset = data.metadata['name']
pn.panel(tool)

In [ ]:
new_dataset = tool()['datasets'][0]
new_data = quest.api.open_dataset(new_dataset)
new_data.index = new_data.index.to_timestamp()
new_data.head()

In [ ]:
parameter = quest.api.get_metadata(new_dataset)[new_dataset]['parameter']
new_data[parameter].hvplot()

# Query Quest for Available Data Providers

In [ ]:
publisher_selector = quest.util.PublisherSelector(default='pub://cuahsi-hydroshare:hs_pub')
pn.panel(publisher_selector)

# Publish New Dataset to Data Depot

In [ ]:
publisher = publisher_selector.value
publish_options = quest.api.get_publish_options(publisher, fmt='param')[publisher]
publish_options.datasets = list()

In [ ]:
pn.panel(publish_options)

In [ ]:
if publish_options.datasets:
    if not quest.api.get_auth_status('cuahsi-hydroshare'):
        quest.api.authenticate_provider('cuahsi-hydroshare')
    hs_resource_id = quest.api.publish(publisher, options=publish_options)
    print('Hydroshare Resource ID:', hs_resource_id)